# Rocket Flight Simulation with RK4, 2D 3DOF, and Pyatmos

This notebook simulates a rocket flight in two dimensions using the RK4 integration scheme. It accounts for:

- A thrust curve loaded from a folder of predefined CSV files. Each CSV file contains two columns: time (s) and thrust (N). The file name (minus `.csv`) is used as the motor name.
- Variable rocket parameters (mass, geometry, etc.)
- Drag (using the drag equation with user-specified diameter, drag coefficient, etc.)
- A standard atmosphere model from **pyatmos** (for density, temperature, and speed of sound vs altitude)
- Wind effects and a fixed launch tilt angle

The state vector is defined as:

\[
\mathbf{x}(t) = \begin{bmatrix} x(t) \\ y(t) \\ v_x(t) \\ v_y(t) \\ m(t) \end{bmatrix}
\]

and the equations of motion are given in the notebook. The interactive UI lets the user choose a motor and customize the rocket parameters.

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import pandas as pd
import glob
from pyatmos import coesa76

# Physical constant
g = 9.80665  # gravity in m/s^2

In [18]:
import pandas as pd
from scipy.interpolate import interp1d
import glob

def load_thrust_curve_from_csv(filename):
    data = pd.read_csv(filename)
    return data['Time (s)'], data[' Thrust (N)']

# Look for all CSV files in the current directory
motor_files = glob.glob("*.csv")

motors = {}
for file in motor_files:
    motor_name = file.replace(".csv", "")
    t_time, t_values = load_thrust_curve_from_csv(file)
    # Create an interpolated thrust function from the data
    thrust_func = interp1d(t_time, t_values, fill_value=0, bounds_error=False)
    # Add motor information to the dictionary
    motors[motor_name] = {
        'thrust_time': t_time,
        'thrust_values': t_values,
        'thrust_func': thrust_func,
        'burn_time': t_time.iloc[-1] if hasattr(t_time, 'iloc') else t_time[-1],
        'Isp': 250  # default Isp (adjust as needed)
    }

if not motors:
    print("No motor CSV files found in the current directory. Please add your thrust curve CSV files.")
else:
    print(f"Loaded motors: {list(motors.keys())}")


Loaded motors: ['o5500', 'm685']


In [44]:
#############################
# Atmosphere model using pyatmos
#############################

def atmosphere_properties(altitude):
    """Return density (kg/m^3), temperature (K), and speed of sound (m/s) at a given altitude (m)."""
    # pyatmos.atmosphere expects altitude in meters and returns a dict with keys 'density', 'temperature', 'speed_of_sound'
    props = coesa76(altitude)
    a = np.sqrt(1.4*287*props.T)
    return props.rho, props.T, a

In [45]:
#############################
# RK4 integrator
#############################

def rk4_step(f, t, y, dt, *args):
    k1 = f(t, y, *args)
    k2 = f(t + dt/2, y + dt*k1/2, *args)
    k3 = f(t + dt/2, y + dt*k2/2, *args)
    k4 = f(t + dt, y + dt*k3, *args)
    return y + dt*(k1 + 2*k2 + 2*k3 + k4)/6

In [46]:
#############################
# Dynamics function for 2D 3DOF rocket
#############################

def dynamics(t, state, motor, rocket_params, sim_params):
    """
    state: [x, y, vx, vy, m]
    motor: dictionary with 'thrust_func', 'Isp', 'burn_time'
    rocket_params: dictionary with parameters such as tilt angle (theta in radians), diameter, Cd, etc.
    sim_params: dictionary with additional parameters (e.g., wind velocity: [v_wx, v_wy])
    """
    x, y, vx, vy, m = state
    
    # Get thrust (if within burn time)
    if t <= motor['burn_time']:
        T = motor['thrust_func'](t)
    else:
        T = 0
    
    # Launch tilt angle
    theta = rocket_params['tilt']
    
    # Wind velocity
    v_wx, v_wy = sim_params['wind']
    
    # Thrust vector components
    T_x = T * np.cos(theta)
    T_y = T * np.sin(theta)
    
    # Relative velocity (rocket velocity relative to wind)
    v_rel_x = vx - v_wx
    v_rel_y = vy - v_wy
    v_rel = np.array([v_rel_x, v_rel_y])
    v_rel_mag = np.linalg.norm(v_rel)
    
    # Get atmospheric properties at current altitude
    dens, temp, a_sound = atmosphere_properties(max(y, 0))
    
    # Drag calculation
    A = np.pi * (rocket_params['diameter']/2)**2
    Cd = rocket_params['Cd']
    D = -0.5 * dens * Cd * A * v_rel_mag * v_rel  # vector drag
    
    # Mass flow rate (only if thrust is applied)
    if T > 0:
        dm_dt = -T / (motor['Isp'] * g)
    else:
        dm_dt = 0
    
    # Accelerations
    ax = (T_x + D[0]) / m
    ay = (T_y + D[1]) / m - g
    
    return np.array([vx, vy, ax, ay, dm_dt])

In [47]:
#############################
# Simulation function
#############################

def run_simulation(motor, rocket_params, sim_params, t_max=10, dt=0.01):
    """
    Runs the simulation and returns a dictionary of time series results.
    """
    # Initial conditions: start at (0,0) with no velocity, and initial mass from rocket parameters
    state = np.array([0.0, 0.0, 0.0, 0.0, rocket_params['initial_mass']])
    
    t_series = []
    x_series = []
    y_series = []
    vx_series = []
    vy_series = []
    m_series = []
    drag_series = []
    q_series = []
    mach_series = []
    total_v_series = []
    
    t = 0
    while t <= t_max and state[1] >= 0:
        t_series.append(t)
        x_series.append(state[0])
        y_series.append(state[1])
        vx_series.append(state[2])
        vy_series.append(state[3])
        m_series.append(state[4])
        
        # Relative velocity
        v_rel = np.array([state[2] - sim_params['wind'][0], state[3] - sim_params['wind'][1]])
        v_rel_mag = np.linalg.norm(v_rel)
        total_v = np.linalg.norm(state[2:4])
        total_v_series.append(total_v)
        
        # Atmospheric properties
        dens, temp, a_sound = atmosphere_properties(max(state[1], 0))
        
        # Drag magnitude
        A = np.pi * (rocket_params['diameter']/2)**2
        drag = 0.5 * dens * rocket_params['Cd'] * A * v_rel_mag**2
        drag_series.append(drag)
        
        # Dynamic pressure
        q = 0.5 * dens * v_rel_mag**2
        q_series.append(q)
        
        # Mach number
        mach = v_rel_mag / a_sound if a_sound > 0 else 0
        mach_series.append(mach)
        
        # RK4 integration step
        state = rk4_step(dynamics, t, state, dt, motor, rocket_params, sim_params)
        t += dt
    
    return {
        'time': np.array(t_series),
        'x': np.array(x_series),
        'y': np.array(y_series),
        'vx': np.array(vx_series),
        'vy': np.array(vy_series),
        'mass': np.array(m_series),
        'drag': np.array(drag_series),
        'dynamic_pressure': np.array(q_series),
        'mach': np.array(mach_series),
        'total_velocity': np.array(total_v_series)
    }

In [48]:
#############################
# Plotting function
#############################

def plot_results(results):
    time = results['time']
    
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 3, 1)
    plt.plot(time, results['y'])
    plt.xlabel('Time (s)')
    plt.ylabel('Altitude (m)')
    plt.title('Altitude vs Time')
    plt.grid(True)
    
    plt.subplot(2, 3, 2)
    plt.plot(time, results['total_velocity'])
    plt.xlabel('Time (s)')
    plt.ylabel('Velocity (m/s)')
    plt.title('Velocity vs Time')
    plt.grid(True)
    
    plt.subplot(2, 3, 3)
    plt.plot(time, results['drag'])
    plt.xlabel('Time (s)')
    plt.ylabel('Drag (N)')
    plt.title('Drag vs Time')
    plt.grid(True)
    
    plt.subplot(2, 3, 4)
    plt.plot(time, results['mass'])
    plt.xlabel('Time (s)')
    plt.ylabel('Mass (kg)')
    plt.title('Mass vs Time')
    plt.grid(True)
    
    plt.subplot(2, 3, 5)
    plt.plot(time, results['dynamic_pressure'])
    plt.xlabel('Time (s)')
    plt.ylabel('Dynamic Pressure (Pa)')
    plt.title('Dynamic Pressure vs Time')
    plt.grid(True)
    
    plt.subplot(2, 3, 6)
    plt.plot(time, results['mach'])
    plt.xlabel('Time (s)')
    plt.ylabel('Mach Number')
    plt.title('Mach Number vs Time')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

In [49]:
#############################
# Interactive interface
#############################

def run_interface():
    # Motor selection widget using the keys from our motors dictionary
    motor_dropdown = widgets.Dropdown(
        options=list(motors.keys()),
        value=list(motors.keys())[0] if motors else None,
        description='Motor:'
    )
    
    # Rocket parameters widgets
    tilt_slider = widgets.FloatSlider(
        value=np.deg2rad(85), min=0, max=np.pi/2, step=0.01,
        description='Tilt (rad)'
    )
    
    diameter_text = widgets.FloatText(
        value=0.1, description='Diameter (m):'
    )
    
    Cd_text = widgets.FloatText(
        value=0.75, description='Cd:'
    )
    
    initial_mass_text = widgets.FloatText(
        value=5.0, description='Initial Mass (kg):'
    )
    
    # Wind velocity widgets (m/s)
    wind_x_text = widgets.FloatText(
        value=0.0, description='Wind X (m/s):'
    )
    wind_y_text = widgets.FloatText(
        value=0.0, description='Wind Y (m/s):'
    )
    
    # Simulation time widget
    tmax_text = widgets.FloatText(
        value=10.0, description='t_max (s):'
    )
    
    # Run simulation button
    run_button = widgets.Button(description='Run Simulation')
    
    out = widgets.Output()
    
    def on_run_button_clicked(b):
        with out:
            out.clear_output()
            # Gather parameters
            selected_motor = motors[motor_dropdown.value]
            
            rocket_params = {
                'tilt': tilt_slider.value,
                'diameter': diameter_text.value,
                'Cd': Cd_text.value,
                'initial_mass': initial_mass_text.value
            }
            
            sim_params = {
                'wind': [wind_x_text.value, wind_y_text.value]
            }
            
            print(f"Running simulation with {motor_dropdown.value}...")
            results = run_simulation(selected_motor, rocket_params, sim_params, t_max=tmax_text.value, dt=0.01)
            plot_results(results)
    
    run_button.on_click(on_run_button_clicked)
    
    ui = VBox([
        HBox([motor_dropdown, tilt_slider]),
        HBox([diameter_text, Cd_text, initial_mass_text]),
        HBox([wind_x_text, wind_y_text, tmax_text]),
        run_button,
        out
    ])
    
    display(ui)

# Run the interface
run_interface()

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import glob
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display

# Import pyatmos for the standard atmosphere model
try:
    import pyatmos
except ImportError:
    raise ImportError("pyatmos is required. Please install it via pip (pip install pyatmos)")

# Gravity constant (m/s^2)
g = 9.80665

#############################
# Motor definitions from CSV files
#############################

def load_thrust_curve_from_csv(filename):
    # Read CSV using pandas and convert columns to numpy arrays
    data = pd.read_csv(filename)
    return data['Time (s)'].values, data[' Thrust (N)'].values

# Look for all CSV files in the current directory
motor_files = glob.glob("*.csv")
motors = {}
for file in motor_files:
    motor_name = file.replace(".csv", "")
    t_time, t_values = load_thrust_curve_from_csv(file)
    if t_time is not None:
        thrust_func = interp1d(t_time, t_values, fill_value=0, bounds_error=False)
        motors[motor_name] = {
            'thrust_time': t_time,
            'thrust_values': t_values,
            'thrust_func': thrust_func,
            'burn_time': t_time[-1],
            'Isp': 250  # default specific impulse (not used for propellant burn here)
        }

if not motors:
    print("No motor CSV files found in the current directory. Please add your thrust curve CSV files.")
else:
    print(f"Loaded motors: {list(motors.keys())}")

#############################
# Atmosphere model using pyatmos
#############################

def atmosphere_properties(altitude):
    """
    Return density (kg/m³), temperature (K), and speed of sound (m/s) at a given altitude (m)
    using the COESA76 model.
    """
    props = pyatmos.coesa76(altitude / 1000)
    # Speed of sound: a = sqrt(gamma * R * T), with gamma=1.4 and R=287 J/(kg·K)
    a = np.sqrt(1.4 * 287 * props.T)
    return props.rho, props.T, a

#############################
# RK4 integrator
#############################

def rk4_step(f, t, y, dt, *args):
    k1 = f(t, y, *args)
    k2 = f(t + dt/2, y + dt*k1/2, *args)
    k3 = f(t + dt/2, y + dt*k2/2, *args)
    k4 = f(t + dt, y + dt*k3, *args)
    return y + dt*(k1 + 2*k2 + 2*k3 + k4)/6

#############################
# Dynamics function for 2D 3DOF rocket
#############################

def dynamics(t, state, motor, rocket_params, sim_params):
    """
    state: [x, y, vx, vy, m]
    motor: dictionary with 'thrust_func' and 'burn_time'
    rocket_params: dictionary with parameters such as tilt (in degrees off vertical), diameter, Cd, dry_mass, and propellant_mass.
    sim_params: dictionary with additional parameters (e.g., wind velocity: [v_wx, v_wy])
    """
    x, y, vx, vy, m = state

    # Thrust is provided only during the burn period.
    if t <= motor['burn_time']:
        T = motor['thrust_func'](t)
        # Propellant burns linearly over burn_time
        dm_dt = -rocket_params['propellant_mass'] / motor['burn_time']
    else:
        T = 0
        dm_dt = 0

    # Tilt is now given in degrees (off vertical) so convert to radians.
    # 0° means perfectly vertical (T_y = T, T_x = 0)
    theta_rad = np.deg2rad(rocket_params['tilt'])
    # With tilt off vertical, vertical component is T*cos(theta) and horizontal is T*sin(theta)
    T_x = T * np.sin(theta_rad)
    T_y = T * np.cos(theta_rad)

    # Wind velocity components
    v_wx, v_wy = sim_params['wind']

    # Relative velocity (rocket velocity relative to wind)
    v_rel = np.array([vx - v_wx, vy - v_wy])
    v_rel_mag = np.linalg.norm(v_rel)

    # Get atmospheric properties at current altitude (ensure altitude >= 0)
    dens, temp, a_sound = atmosphere_properties(max(y, 0))

    # Drag calculation using unit vector of relative velocity
    A = np.pi * (rocket_params['diameter'] / 2)**2
    Cd = rocket_params['Cd']
    if v_rel_mag > 0:
        D = -0.5 * dens * Cd * A * v_rel_mag**2 * (v_rel / v_rel_mag)
    else:
        D = np.array([0.0, 0.0])

    # Compute accelerations
    ax = (T_x + D[0]) / m
    ay = (T_y + D[1]) / m - g

    return np.array([vx, vy, ax, ay, dm_dt])

#############################
# Simulation function
#############################

def run_simulation(motor, rocket_params, sim_params, t_max=10, dt=0.01):
    """
    Runs the simulation for a fixed time (t_max) and returns time series results.
    The initial mass is the sum of the dry mass and the propellant mass.
    Also records temperature and speed of sound at each time step.
    """
    initial_mass = rocket_params['dry_mass'] + rocket_params['propellant_mass']
    state = np.array([0.0, 0.0, 0.0, 0.0, initial_mass])
    t_series = []
    x_series = []
    y_series = []
    vx_series = []
    vy_series = []
    m_series = []
    drag_series = []
    q_series = []
    mach_series = []
    vertical_velocity_series = []  # vertical velocity (vy) with sign
    acc_series = []  # vertical acceleration (signed)
    temp_series = []  # temperature at current altitude
    a_series = []     # speed of sound at current altitude

    t = 0
    while t <= t_max:
        t_series.append(t)
        x_series.append(state[0])
        y_series.append(state[1])
        vx_series.append(state[2])
        vy_series.append(state[3])
        m_series.append(state[4])
        vertical_velocity_series.append(state[3])
        
        # Compute atmospheric properties at current altitude
        dens, temp, a_sound = atmosphere_properties(max(state[1], 0))
        temp_series.append(temp)
        a_series.append(a_sound)
        
        # Relative velocity
        v_rel = np.array([state[2] - sim_params['wind'][0], state[3] - sim_params['wind'][1]])
        v_rel_mag = np.linalg.norm(v_rel)
        
        # Drag magnitude (for plotting)
        A = np.pi * (rocket_params['diameter'] / 2)**2
        drag = 0.5 * dens * rocket_params['Cd'] * A * v_rel_mag**2
        drag_series.append(drag)
        
        # Dynamic pressure
        q = 0.5 * dens * v_rel_mag**2
        q_series.append(q)
        
        # Mach number
        mach = v_rel_mag / a_sound if a_sound > 0 else 0
        mach_series.append(mach)
        
        # Compute vertical acceleration (signed) from dynamics (use ay)
        deriv = dynamics(t, state, motor, rocket_params, sim_params)
        acc = deriv[3]  # vertical acceleration (can be negative)
        acc_series.append(acc)
        
        state = rk4_step(dynamics, t, state, dt, motor, rocket_params, sim_params)
        t += dt

    return {
        'time': np.array(t_series),
        'x': np.array(x_series),
        'y': np.array(y_series),
        'vx': np.array(vx_series),
        'vy': np.array(vy_series),
        'mass': np.array(m_series),
        'drag': np.array(drag_series),
        'dynamic_pressure': np.array(q_series),
        'mach': np.array(mach_series),
        'vertical_velocity': np.array(vertical_velocity_series),
        'acceleration': np.array(acc_series),
        'temperature': np.array(temp_series),
        'speed_of_sound': np.array(a_series)
    }

#############################
# Plotting function
#############################

def get_end_time(results):
    end_time = 0
    time = results['time']
    alt = results['y']

    for i in range(10, len(time)):
        if (alt[i] < 3):
            end_time = i

    return end_time

def plot_results(results):
    time = results['time']
    alt = results['y']
    
    # Compute density for each altitude using the atmosphere model
    densities = np.array([atmosphere_properties(max(a, 0))[0] for a in alt])
    
    # Set up a 3x3 grid of subplots
    plt.figure(figsize=(14, 10))
    
    # Subplot 1: Altitude vs Time
    plt.subplot(3, 3, 1)
    plt.plot(time, alt)
    plt.xlabel('Time (s)')
    plt.ylabel('Altitude (m)')
    plt.title('Altitude vs Time')
    plt.grid(True)
    
    # Subplot 2: Vertical Velocity vs Time
    plt.subplot(3, 3, 2)
    plt.plot(time, results['vertical_velocity'])
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Velocity (m/s)')
    plt.title('Vertical Velocity vs Time')
    plt.grid(True)
    
    # Subplot 3: Drag vs Time
    plt.subplot(3, 3, 3)
    plt.plot(time, results['drag'])
    plt.xlabel('Time (s)')
    plt.ylabel('Drag (N)')
    plt.title('Drag vs Time')
    plt.grid(True)
    
    # Subplot 4: Mass vs Time
    plt.subplot(3, 3, 4)
    plt.plot(time, results['mass'])
    plt.xlabel('Time (s)')
    plt.ylabel('Mass (kg)')
    plt.title('Mass vs Time')
    plt.grid(True)
    
    # Subplot 5: Dynamic Pressure vs Time
    plt.subplot(3, 3, 5)
    plt.plot(time, results['dynamic_pressure'])
    plt.xlabel('Time (s)')
    plt.ylabel('Dynamic Pressure (Pa)')
    plt.title('Dynamic Pressure vs Time')
    plt.grid(True)
    
    # Subplot 6: Mach Number vs Time
    plt.subplot(3, 3, 6)
    plt.plot(time, results['mach'])
    plt.xlabel('Time (s)')
    plt.ylabel('Mach Number')
    plt.title('Mach Number vs Time')
    plt.grid(True)
    
    # Subplot 7: Vertical Acceleration vs Time (signed)
    plt.subplot(3, 3, 7)
    plt.plot(time, results['acceleration'])
    plt.xlabel('Time (s)')
    plt.ylabel('Vertical Acceleration (m/s²)')
    plt.title('Vertical Acceleration vs Time')
    plt.grid(True)
    
    # Subplot 8: Density vs Altitude
    plt.subplot(3, 3, 8)
    plt.plot(alt, densities)
    plt.xlabel('Altitude (m)')
    plt.ylabel('Density (kg/m³)')
    plt.title('Density vs Altitude')
    plt.grid(True)
    
    # Subplot 9: Temperature and Speed of Sound vs Altitude
    plt.subplot(3, 3, 9)
    plt.plot(alt, results['temperature'], label='Temperature (K)')
    plt.plot(alt, results['speed_of_sound'], label='Speed of Sound (m/s)')
    plt.xlabel('Altitude (m)')
    plt.ylabel('Value')
    plt.title('Temp & Speed of Sound vs Altitude')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

#############################
# Interactive interface
#############################

def run_interface():
    motor_dropdown = widgets.Dropdown(
        options=list(motors.keys()),
        value=list(motors.keys())[0] if motors else None,
        description='Motor:'
    )
    
    tilt_slider = widgets.FloatSlider(
        value=0, min=0, max=90, step=1,
        description='Tilt (° off vertical)'
    )
    
    diameter_text = widgets.FloatText(
        value=0.1, description='Diameter (m):'
    )
    
    Cd_text = widgets.FloatText(
        value=0.75, description='Cd:'
    )
    
    dry_mass_text = widgets.FloatText(
        value=10.0, description='Dry Mass (kg):'
    )
    
    propellant_mass_text = widgets.FloatText(
        value=5.0, description='Propellant Mass (kg):'
    )
    
    wind_x_text = widgets.FloatText(
        value=0.0, description='Wind X (m/s):'
    )
    wind_y_text = widgets.FloatText(
        value=0.0, description='Wind Y (m/s):'
    )
    
    tmax_text = widgets.FloatText(
        value=100.0, description='t_max (s):'
    )
    
    run_button = widgets.Button(description='Run Simulation')
    out = widgets.Output()
    
    def on_run_button_clicked(b):
        with out:
            out.clear_output()
            selected_motor = motors[motor_dropdown.value]
            rocket_params = {
                'tilt': tilt_slider.value,
                'diameter': diameter_text.value,
                'Cd': Cd_text.value,
                'dry_mass': dry_mass_text.value,
                'propellant_mass': propellant_mass_text.value
            }
            sim_params = {
                'wind': [wind_x_text.value, wind_y_text.value]
            }
            print(f"Running simulation with {motor_dropdown.value}...")
            sim_time = tmax_text.value
            results = run_simulation(selected_motor, rocket_params, sim_params, t_max=sim_time, dt=0.01)
            plot_results(results)
            end_time = get_end_time(results)
            if (end_time < tmax_text.value):
                sim_time = end_time
            plt.xlim(0, end_time)
    
    run_button.on_click(on_run_button_clicked)
    
    ui = VBox([
        HBox([motor_dropdown, tilt_slider]),
        HBox([diameter_text, Cd_text, dry_mass_text, propellant_mass_text]),
        HBox([wind_x_text, wind_y_text, tmax_text]),
        run_button,
        out
    ])
    
    display(ui)

if __name__ == "__main__":
    run_interface()

Loaded motors: ['o5500', 'm685']
